In [185]:
import pandas as pd
import numpy as np


In [186]:
rawdf = pd.read_csv('GreenspaceDownload/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.csv', encoding='unicode_escape', low_memory=False)

In [187]:
rawdf.shape

(36902, 160)

In [188]:
print(list(rawdf.columns))

['ID_HDC_G0', 'QA2_1V', 'AREA', 'BBX_LATMN', 'BBX_LONMN', 'BBX_LATMX', 'BBX_LONMX', 'GCPNT_LAT', 'GCPNT_LON', 'CTR_MN_NM', 'CTR_MN_ISO', 'XBRDR', 'XCTR_NBR', 'XC_NM_LST', 'XC_ISO_LST', 'GRGN_L1', 'GRGN_L2', 'UC_NM_MN', 'UC_NM_LST', 'UC_NM_SRC', 'H75_NBR', 'H90_NBR', 'H00_NBR', 'H75_AREA', 'H90_AREA', 'H00_AREA', 'E_BM_NM_LST', 'E_SL_LST', 'EL_AV_ALS', 'E_KG_NM_LST', 'E_RB_NM_LST', 'E_WR_P_90', 'E_WR_P_00', 'E_WR_P_14', 'E_WR_T_90', 'E_WR_T_00', 'E_WR_T_14', 'B75', 'B90', 'B00', 'B15', 'P75', 'P90', 'P00', 'P15', 'BUCAP75', 'BUCAP90', 'BUCAP00', 'BUCAP15', 'NTL_AV', 'GDP90_SM', 'GDP00_SM', 'GDP15_SM', 'INCM_CMI', 'DEV_CMI', 'TT2CC', 'E_GR_AV90', 'E_GR_AV00', 'E_GR_AV14', 'E_GR_AH90', 'E_GR_AM90', 'E_GR_AL90', 'E_GR_AT90', 'E_GR_AH00', 'E_GR_AM00', 'E_GR_AL00', 'E_GR_AT00', 'E_GR_AH14', 'E_GR_AM14', 'E_GR_AL14', 'E_GR_AT14', 'E_EC2E_E75', 'E_EC2E_E90', 'E_EC2E_E00', 'E_EC2E_E15', 'E_EC2E_R75', 'E_EC2E_R90', 'E_EC2E_R00', 'E_EC2E_R15', 'E_EC2E_I75', 'E_EC2E_I90', 'E_EC2E_I00', 'E_EC2E_I15

In [189]:
cols_to_keep = ['GCPNT_LAT', 'GCPNT_LON', 'CTR_MN_NM', 'UC_NM_MN', 'UC_NM_LST', 'E_GR_AV14', 'E_GR_AT14', 'SDG_A2G14', 'SDG_OS15MX', 'P15', 'B15', 'BUCAP15', 'INCM_CMI', 'DEV_CMI', 'GDP15_SM', 'E_BM_NM_LST', 'E_WR_T_14']
df = rawdf[cols_to_keep]
df = df[df['CTR_MN_NM'] == 'United States']
df.replace(to_replace=['?', '??', '???', 'NAN'], value = [np.nan, np.nan, np.nan, np.nan], inplace=True)
df.rename(columns={'GCPNT_LAT': 'Latitude', 'GCPNT_LON': 'Longitude', 'CTR_MN_NM': 'Country', 'UC_NM_MN': 'Urban Center', 'UC_NM_LST': 'Cities in Urban Center'}, inplace=True)
df.head()

,Latitude,Longitude,Country,Urban Center,Cities in Urban Center,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,P15,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14
0,21.340678,-157.893497,United States,Honolulu,Honolulu; Waipahu; Pearl City; Aiea,0.369290388,183.811667,0.226415,56.41,512853.66670,80.647377,157.252219,HIC,MDR,2.192668e+10,Tropical and Subtropical Dry Broadleaf Forests,23.5266223
2,34.923123,-120.434372,United States,Santa Maria,Santa Maria,0.312845526,54.450694,0.040129,23.64,123181.28480,42.000805,340.967420,HIC,MDR,4.174295e+09,"Mediterranean Forests, Woodlands, and Scrub",14.71819115
3,36.607720,-121.882378,United States,Monterey,Monterey,0.358100922,47.815404,0.138683,42.17,67772.28886,27.759470,409.599122,HIC,MDR,1.998786e+09,"Mediterranean Forests, Woodlands, and Scrub",15.27411008
4,34.427664,-119.743693,United States,Santa Barbara,Santa Barbara,0.362784899,59.576284,0.061348,36.5,114753.15020,38.101749,332.032274,HIC,MDR,4.159702e+09,"Mediterranean Forests, Woodlands, and Scrub",15.37690687
5,36.971274,-121.978684,United States,Santa Cruz,Santa Cruz,0.353734305,56.095926,0.109287,39.46,101498.14560,34.507401,339.980601,HIC,MDR,3.890861e+09,"Mediterranean Forests, Woodlands, and Scrub",15.27411008


In [190]:
a = df.duplicated(subset='Urban Center')
dupind = list(a.index[a])
print(dupind)
duplicates = df[df.index.isin(dupind)]
dupcities = (duplicates['Urban Center'].unique()).tolist()
print(dupcities)

[524, 626, 649, 728, 876, 909, 1003, 1028, 1044]
['Springfield', 'Bloomington', 'Lafayette', 'Columbus', 'Columbia', 'Rochester', 'Lawrence', 'Portland']


In [191]:
states = pd.read_csv('uscities.csv')
states = states[['city', 'state_id', 'lat', 'lng']]
states.rename(columns={'city': 'Urban Center', 'state_id': 'State'}, inplace=True)
print(states.shape)
states = states.drop(states[states['Urban Center'].isin(dupcities)].index)
print(states.shape)

(31120, 4)
(31019, 4)


In [192]:
df = df.join(states.set_index('Urban Center'), on='Urban Center', how='left')

In [193]:
df[df.index.duplicated()].index.unique()

Index([   2,    3,    5,    6,    9,   12,   13,   16,   17,   21,
       ...
        996, 1008, 1011, 1015, 1019, 1021, 1022, 1023, 1024, 1027],
      dtype='int64', length=189)

In [128]:
#df[df['State'].isna()]

In [129]:
# df.at[20, 'State'] = 'CA'
# df.at[381, 'State'] = 'KS'
# df.at[482, 'Urban Center'] = "O'Fallon"
# df.at[482, 'State'] = 'MO'
# df.at[550, 'State'] = 'MN'
# df.at[553, 'Urban Center'] = 'Minneapolis'
# df.at[553, 'State'] = 'MN'
# df.at[763, 'Urban Center'] = 'Ypsilanti'
# df.at[763, 'State'] = 'MI'
# df.at[819, 'State'] = 'PA'
# df.at[854, 'State'] = 'VA'
# df.at[941, 'State'] = 'NJ'
# df.at[948, 'State'] = 'NJ'

# df[df['State'].isna()]

In [107]:
temp = df[df['Urban Center'] == "Portland"]
temp

,Latitude,Longitude,Country,Urban Center,Cities in Urban Center,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,P15,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14,State
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,OR
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,ME
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,TX
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,TN
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,IN
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,MI
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,PA
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,ND
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,AR
100,45.530949,-122.665733,United States,Portland,Portland; Vancouver,0.490659077,947.669878,0.600467,60.87,1.655686e+06,660.366943,398.847902,HIC,MDR,4.767102e+10,Temperate Broadleaf and Mixed Forests,11.73069286,CO


Convert ??? and NAN to NaN, filter to United States, get states and deal with duplicates, remove rows with missing data